In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "Dakota123"
shelter = AnimalShelter("accuser", "Dakota123")


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable')

app.head = [
    html.Link(
    href='https://learn.snhu.edu/content/enforced/1069832-CS-340-T5626-OL-TRAD-UG.22EW5/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=w3rTclyxYnuL4Py75ebGMO7zw&ou=1069832',
    rel='icon'),]
#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(html.Img(src='https://learn.snhu.edu/content/enforced/1069832-CS-340-T5626-OL-TRAD-UG.22EW5/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=w3rTclyxYnuL4Py75ebGMO7zw&ou=1069832',
                     style={'height':'5', 'width':'5'})),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Chris Dechene - 6/17/2022'))),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='radio-button',
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain', 'value': 'mountain'},
                {'label': 'Disaster', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            labelStyle={'display': 'inline-block'}
        ),
        html.Div(id='dd-output-container')
    ]),
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('datatable-id', 'data'),
    [Input('radio-button', 'value')])
def datatable_filter_update(value):
    if value == 'water':
        df = pd.DataFrame.from_records(shelter.read({ "breed": { "$in": [ "Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland" ] }, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks":{ "$gte":26, "$lte":156 } }))
    elif value == 'mountain':
        df = pd.DataFrame.from_records(shelter.read({ "breed": { "$in": [ "German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler" ] }, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks":{ "$gte":26, "$lte":156 } }))
    elif value == 'disaster':
        df = pd.DataFrame.from_records(shelter.read({ "breed": { "$in": [ "Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler" ] }, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks":{ "$gte":20, "$lte":300 } }))
    else:
        df = pd.DataFrame.from_records(shelter.read_all({}))
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def generate_chart(names, values):
    dff =pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, values=values, names=names, hole=.3)
    return fig

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app